In [1]:
from games_setup import *
import SBMLLint.common.constants as cn
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
import collections
import tesbml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
data_dir=cn.BIOMODELS_DIR
#files = [f for f in os.listdir(data_dir) if f[-4:] == ".xml"]
files = [f for f in os.listdir(data_dir) if f[:7] == "BIOMD00"]
paths = [os.path.join(data_dir, filename) for filename in files]
unloadable_models = ["BIOMD0000000075_url.xml",
                     "BIOMD0000000081_url.xml",
                     "BIOMD0000000094_url.xml",
                     "BIOMD0000000353_url.xml",
                     "BIOMD0000000596_url.xml"]
net_files = list(set(files) - set(unloadable_models))
len(net_files)

651

In [3]:
len(files)

656

In [4]:
# statistics columns
NUM_REACTIONS = "num_reactions(nonbdry)"
LP_ERROR = "lp_error"
GAMES_ERROR = "games_error"
GAMESPP_ERROR = "gamespp_error"
TYPEI_ERROR = "type1_error"
TYPEII_ERROR = "type2_error"
CANCELING_ERROR = "canceling_error"
ECHELON_ERROR = "echelon_error"
TYPEIII_ERROR = "type3_error"
result_columns = [NUM_REACTIONS,
                  LP_ERROR,
                  GAMES_ERROR,
                  GAMESPP_ERROR,
                  TYPEI_ERROR,
                  TYPEII_ERROR,
                  CANCELING_ERROR,
                  ECHELON_ERROR,
                  TYPEIII_ERROR]

In [5]:
results = pd.DataFrame(0, index=files, columns=result_columns)
results[:5]

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000199_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000189_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000387_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000397_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000413_url.xml,0,0,0,0,0,0,0,0,0


In [6]:
simple = SimpleSBML()
simple.initialize(os.path.join(data_dir, "BIOMD0000000244_url.xml"))
s = StoichiometryMatrix(simple)
consistent = s.isConsistent()
print("consistent? ", consistent)

consistent?  False


/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_util.py:704: OptimizeWarning:

A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.



In [6]:
# LP only
simple = SimpleSBML()
count = 0
lp_start = time.time()
for file in net_files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    s = StoichiometryMatrix(simple)
    num_reactions = s.stoichiometry_matrix.shape[1]
    results.at[file, NUM_REACTIONS] = num_reactions
    if num_reactions:
      consistent = s.isConsistent()
    else:
      consistent = -1
    results.at[file, LP_ERROR] = 1 - int(consistent)
  except:
    results.at[file, LP_ERROR] = -1
lp_end = time.time()
lp_time = lp_end - lp_start
print("Analysis finished!")
print("LP time:", lp_time)

/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_util.py:704: OptimizeWarning:

A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.

/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_simplex.py:201: OptimizeWarning:

The pivot operation produces a pivot value of: 2.7e-12, which is only slightly greater than the specified tolerance 1.0e-12. This may lead to issues regarding the numerical stability of the simplex method. Removing redundant constraints, changing the pivot strategy via Bland's rule or increasing the tolerance may help reduce the issue.



we are analyzing Model number: 100
we are analyzing Model number: 200
we are analyzing Model number: 300
we are analyzing Model number: 400
we are analyzing Model number: 500
we are analyzing Model number: 600
Analysis finished!
LP time: 66.5884690284729


In [12]:
lp_results = results[results[LP_ERROR] == 1]
len(lp_results)
print("ISS for LP is:", np.mean(lp_results[NUM_REACTIONS]))

ISS for LP is: 58.675862068965515


In [9]:
len(results[results[LP_ERROR]==1])

143

In [10]:
len(results[results[LP_ERROR]==-1])

2

In [11]:
# GAMES only
simple = SimpleSBML()
count = 0
games_start = time.time()
for file in net_files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    m = GAMES_PP(simple)
    if simple.reactions:
      res = m.analyze(simple_games=True, error_details=False)
      results.at[file, GAMES_ERROR] = int(res)
      if res:
        gr = GAMESReport(m)
        summary = m.error_summary
        if m.type_one_errors:
          results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
          report, error_num = gr.reportTypeOneError(m.type_one_errors, explain_details=True)
        if m.type_two_errors:
          results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
          report, error_num = gr.reportTypeTwoError(m.type_two_errors, explain_details=True)
  except:
    results.at[file, GAMES_ERROR] = -1   
games_end = time.time()
games_time = games_end - games_start
print("Analysis finished!")
print("GAMES time:", games_time)

we are analyzing Model number: 100
we are analyzing Model number: 200
we are analyzing Model number: 300
we are analyzing Model number: 400
we are analyzing Model number: 500
we are analyzing Model number: 600
Analysis finished!
GAMES time: 123.1564610004425


In [12]:
print(games_time)

123.1564610004425


In [13]:
print("number of detected errors: ", len(results[results[GAMES_ERROR]==1]))
print("number of GAMES but not in LP", len(results[(results[GAMES_ERROR]==1) & (results[LP_ERROR]!=1)]))

number of detected errors:  109
number of GAMES but not in LP 0


In [14]:
len(results[results[GAMES_ERROR]==-1])

0

In [7]:
# GAMES+
# file, GAMES_ERROR coding:
# 0; normal - no error found
# -1; not loaded or error found
# 1; normal - error found
# 2; echelon error found, but it is not explainable
# 3; type III error found, but it is not explainable
simple = SimpleSBML()
count = 0
gamespp_start = time.time()
for file in net_files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    m = GAMES_PP(simple)
    if simple.reactions:
      res = m.analyze(simple_games=False, error_details=False)
      results.at[file, GAMESPP_ERROR] = int(res)
      if res:
        gr = GAMESReport(m)
        summary = m.error_summary
        if m.type_one_errors:
          results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
          report, error_num = gr.reportTypeOneError(m.type_one_errors, explain_details=True)
        if m.type_two_errors:
          results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
          report, error_num = gr.reportTypeTwoError(m.type_two_errors, explain_details=True)
        if m.canceling_errors:
          results.at[file, CANCELING_ERROR] = len(m.canceling_errors)
          report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
        if m.echelon_errors:
          results.at[file, ECHELON_ERROR] = len(m.echelon_errors)
          report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
          if report is False:
            results.at[file, GAMESPP_ERROR] = 2
        if m.type_three_errors:
          results.at[file, TYPEIII_ERROR] = len(m.type_three_errors)
          report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
          if report is False:
            results.at[file, GAMESPP_ERROR] = 3
  except:
    results.at[file, GAMES_ERROR] = -1   
gamespp_end = time.time()
gamespp_time = gamespp_end - gamespp_start
print("Analysis finished!")
print("GAMES++ time:", gamespp_time)

we are analyzing Model number: 100
we are analyzing Model number: 200
we are analyzing Model number: 300
we are analyzing Model number: 400
we are analyzing Model number: 500
we are analyzing Model number: 600
Analysis finished!
GAMES++ time: 220.54042029380798


In [8]:
print("number of detected errors: ", len(results[results[GAMESPP_ERROR]==1]))
print("number of GAMES errors not in LP", len(results[(results[GAMESPP_ERROR]==1) & (results[LP_ERROR]!=1)]))
len(results[results[GAMESPP_ERROR]==-1])

number of detected errors:  136
number of GAMES errors not in LP 0


0

In [9]:
len(results[results[GAMESPP_ERROR]==2])

1

In [10]:
len(results[results[GAMESPP_ERROR]==3])

3

In [11]:
results[(results[GAMESPP_ERROR]==2) | (results[GAMESPP_ERROR]==3)]

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000628_url.xml,584,0,0,3,0,0,0,22,1
BIOMD0000000574_url.xml,576,0,0,3,0,0,0,79,5
BIOMD0000000475_url.xml,25,0,0,2,0,0,0,1,0
BIOMD0000000243_url.xml,19,1,0,3,0,0,0,0,1


In [24]:
results[(results[GAMESPP_ERROR] != -1)  & (results[LP_ERROR] == -1)]

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000596_url.xml,0,-1,-1,0,0,0,0,0,0
BIOMD0000000496_url.xml,285,-1,0,0,0,0,0,0,0
BIOMD0000000081_url.xml,0,-1,-1,0,0,0,0,0,0
BIOMD0000000075_url.xml,0,-1,-1,0,0,0,0,0,0
BIOMD0000000497_url.xml,285,-1,0,0,0,0,0,0,0
BIOMD0000000094_url.xml,0,-1,-1,0,0,0,0,0,0
BIOMD0000000353_url.xml,0,-1,-1,0,0,0,0,0,0


In [38]:
unloadable_models = ["BIOMD0000000075_url.xml",
                     "BIOMD0000000081_url.xml",
                     "BIOMD0000000094_url.xml",
                     "BIOMD0000000353_url.xml",
                     "BIOMD0000000596_url.xml"]
nonlp_models = ["BIOMD0000000496_url.xml",
               "BIOMD0000000497_url.xml"]

In [39]:
file = "BIOMD0000000496_url.xml"
simple.initialize(os.path.join(data_dir, file))
s = StoichiometryMatrix(simple)
num_reactions = s.stoichiometry_matrix.shape[1]
results.at[file, NUM_REACTIONS] = num_reactions
if num_reactions:
  consistent = s.isConsistent()
else:
  consistent = -1

/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_util.py:704: OptimizeWarning:

A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.



ValueError: Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 2.0e-11 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 2.0e-11. If this tolerance is unacceptably  large the problem may be infeasible.

In [37]:
# # GAMES++ 
# simple = SimpleSBML()
# count = 0
# gamespp_start = time.time()
# for file in files[82:89]:
#   print("We are doing file", file)
#   count += 1
#   if (count%100)==0:
#     print("we are analyzing Model number:", count)
#   simple.initialize(os.path.join(data_dir, file))
#   m = GAMES_PP(simple)
#   if simple.reactions:
#     res = m.analyze(simple_games=False, error_details=False)
#     results.at[file, GAMESPP_ERROR] = int(res)
#     if res:
#       gr = GAMESReport(m)
#       summary = m.error_summary
#       if m.type_one_errors:
#         results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
#         report, error_num = gr.reportTypeOneError(m.type_one_errors)
#       if m.type_two_errors:
#         results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
#         report, error_num = gr.reportTypeTwoError(m.type_two_errors)
#       if m.canceling_errors:
#         results.at[file, CANCELING_ERROR] = len(m.canceling_errors)
#         report, error_num = gr.reportCancelingError(m.canceling_errors)
#       if m.echelon_errors:
#         results.at[file, ECHELON_ERROR] = len(m.echelon_errors)
#         report, error_num = gr.reportEchelonError(m.echelon_errors)
#       if m.type_three_errors:
#         results.at[file, TYPEIII_ERROR] = len(m.type_three_errors)
#         report, error_num = gr.reportTypeThreeError(m.type_three_errors)

We are doing file BIOMD0000000032_url.xml
We are doing file BIOMD0000000022_url.xml
We are doing file BIOMD0000000479_url.xml
We are doing file BIOMD0000000501_url.xml
We are doing file BIOMD0000000469_url.xml
We are doing file BIOMD0000000511_url.xml
We are doing file BIOMD0000000354_url.xml
